In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import time
time_start=time.time()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import matplotlib.pyplot as plt
from torch import optim
from sklearn import linear_model 
from sklearn.preprocessing import MinMaxScaler 

import datetime
nowTime0 = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间

torch.set_default_tensor_type(torch.DoubleTensor)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# read data and spliting to generate training samples

def read_data(prob_No, window_size, stride, MaxRUL):
    '''
    :param prob_No: 1,2,3,4,  ----representing FD001 ~ FD004
    :param window_size: slide window sizes
    :param stride: slide step length
    : return training & testing set data (dims: num x  lens/or window_size x channels) & labels (dims: num x lens/or window_size) or (dims: num x lens/1)
    ''' 
    train_df = pd.read_csv('../input/cmapssdata/train.csv', usecols=[0,1,2,  3,4,5,   7,8,9,12,13,14,16,17,18,19,20,22,25,26])
    test_df = pd.read_csv('../input/cmapssdata/test.csv', usecols=[0,1,2,  3,4,5,   7,8,9,12,13,14,16,17,18,19,20,22,25,26])
    rul_df = pd.read_csv('../input/cmapssdata/RUL.csv')
    
    # 训练集
    train_data = train_df[(train_df.dataset_id == 'FD00'+str(prob_No))]
    train_data_total = []
    train_label_total = []
    cyclenum1 = train_data.unit_id.max()
    LRmodel = linear_model.LinearRegression()   # 线性回归
    train_cyclen_list = []
    train_sample_num = 0
    
    for cycle in range(cyclenum1):
        data = train_data[(train_data.unit_id == cycle+1)]
        cyclens = data.cycle.max()
        train_cyclen_list.append(cyclens)
        
        data = data.values[:, 6:].astype(float)
        labels = [(cyclens - t if cyclens - t <= MaxRUL else MaxRUL) for t in range(1, cyclens + 1)]
        
        for i in range(0, cyclens, stride):
            x = np.array(list(range(i+1-window_size, i+1, stride)))   # 对time step回归，x为time step list
            if i >= window_size - 1:
                scaled_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(data[i+1-window_size:i+1])
            else:
                padding_data = np.zeros((window_size - i - 1, data[0:i+1].shape[-1]))    # 不足窗口的数据填为0  补充零待考证
                y = np.vstack((padding_data, data[0:i+1]))
                scaled_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(y)
            
            max_data = np.max(scaled_data,axis=0)    # 最大值
            mean_data = np.mean(scaled_data,axis=0)    # 均值
#             std_data = np.std(scaled_data,axis=0,ddof=1)    # 标准差

            coef = 0    # 回归系数
            for sensor in range(scaled_data.shape[-1]):
                LRmodel.fit(x.reshape(-1,1), scaled_data[:, sensor].reshape(-1,1))    # 利用sklearn中的线性回归模型
                coef = np.hstack((coef,LRmodel.coef_[:,0]))   # 获得回归系数

            sample_data = torch.tensor(np.vstack((scaled_data, max_data, mean_data, coef[1:]))) #含最大值，最小值，极差，均值，标准差，回归系数及归一化的时间窗数据
            
            train_data_total.append(sample_data.T)
            train_label_total.append(torch.tensor(labels[i]))
            
            train_sample_num += 1
    
    # 测试集
    test_data = test_df[(test_df.dataset_id == 'FD00'+str(prob_No))]
    test_data_total = []
    test_label_total = []
    cyclenum2 = test_data.unit_id.max()
    test_cyclen_list = []
    test_sample_num = 0
    
    test_label = rul_df[(rul_df.dataset_id == 'FD00'+str(prob_No))]
    test_label = test_label.values[:,2] # 1维
    
    for cycle in range(cyclenum2):
        data = test_data[(test_data.unit_id == cycle+1)]
        cyclens = data.cycle.max()
        test_cyclen_list.append(cyclens)
        
        data = data.values[:, 6:].astype(float)
        labels = [((test_label[cycle] + cyclens - 1 - t) \
                   if (test_label[cycle] + cyclens - 1 - t) <= MaxRUL else MaxRUL) for t in range(cyclens)]
        
        for i in range(0, cyclens, stride):
            x = np.array(list(range(i+1-window_size, i+1, stride)))   # 对time step回归，x为time step list
            if i >= window_size - 1:
                scaled_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(data[i+1-window_size:i+1])
            else:
                padding_data = np.zeros((window_size - i - 1, data[0:i+1].shape[-1]))    # 不足窗口的数据填为0  补充零待考证
                y = np.vstack((padding_data, data[0:i+1]))
                scaled_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(y)
            
            max_data = np.max(scaled_data,axis=0)    # 最大值

            mean_data = np.mean(scaled_data,axis=0)    # 均值
#             std_data = np.std(scaled_data,axis=0,ddof=1)    # 标准差
             
            coef = 0    # 回归系数
            for sensor in range(scaled_data.shape[-1]):
                LRmodel.fit(x.reshape(-1,1), scaled_data[:, sensor].reshape(-1,1))    # 利用sklearn中的线性回归模型
                coef = np.hstack((coef,LRmodel.coef_[:,0]))

            sample_data = torch.tensor(np.vstack((scaled_data, max_data, mean_data, coef[1:])))    #含均值 回归系数的归一化的时间窗数据
            
            test_data_total.append(sample_data.T)
            test_label_total.append(torch.tensor(labels[i]))
            
            test_sample_num += 1
    
    return train_data_total, train_label_total, train_sample_num, train_cyclen_list, test_data_total, test_label_total, test_sample_num, test_cyclen_list

In [ ]:
# Main Function - 1
window_size = 60    # FD001-0004的时间窗大小分别为50,50,50,50
# read data and spliting to generate training samples
nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime)+': 初始化完成，正在读取数据，请稍等......')

train_data, train_label, train_sample_num, train_cyclen_list, test_data, test_label, test_sample_num, test_cyclen_list = read_data(3, window_size, 1, 125)    # 如果maxRUL无limited， 此处可设maxRUL为极大值如10000

nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime)+': 读取数据完成')

In [ ]:
import math
import torch.nn as nn

# SE Block
class SEblock(nn.Module):
    def __init__(self, sensor_num,r):
        super(SEblock, self).__init__()
        ':r 降维率'
        # squeeze
        self.sq = torch.nn.AdaptiveAvgPool1d(1)
        # excitation
        self.fc1= nn.Linear(sensor_num, int(sensor_num/r))
        self.relu = nn.ReLU()
        self.fc2= nn.Linear(int(sensor_num/r), sensor_num)
        self.sigmoid = nn.Sigmoid()
        
    def init_weights(self):
        self.fc1.weight.data.normal_(0, 0.01)
        self.fc2.weight.data.normal_(0, 0.01)

    def forward(self, x):
        torch.set_default_tensor_type(torch.FloatTensor)
        z_c = self.sq(x).transpose(-1,-2)    # B x 1 x C
        s = self.fc1(z_c)    # B x 1 x C/r
        s = self.relu(s)     # B x 1 x C/r
        s = self.fc2(s)      # B x 1 x C
        s = self.sigmoid(s).transpose(-1,-2)  # B x C x 1
        o = s * x   # B x C x L
        return o

In [ ]:
from torch.nn.utils import weight_norm


# 卷积结束后会因为padding导致卷积之后的新数据的尺寸B>输入数据的尺寸A，所以只保留输出数据中前面A个数据
# 修改数据尺寸
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x): 
        return x[:, :, :-self.chomp_size].contiguous()


# Residual Block --- TemporalBlock
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size1,  kernel_size2, kernel_size3, stride, dilation, dropout):
        super(TemporalBlock, self).__init__()
        # 尺度1
        padding1 = (kernel_size1-1) * dilation
        self.conv11 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size1,
                                           stride=stride, padding=padding1, dilation=dilation))
        self.chomp11 = Chomp1d(padding1)
#         self.relu11 = nn.ReLU()
#         self.relu11 = nn.LeakyReLU()
        self.relu11 = nn.PReLU(n_outputs)
        self.dropout11 = nn.Dropout(dropout)
        
        self.conv21 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size1,
                                           stride=stride, padding=padding1, dilation=dilation))
        self.chomp21 = Chomp1d(padding1)
#         self.relu21 = nn.ReLU()
#         self.relu21 = nn.LeakyReLU()
        self.relu21 = nn.PReLU(n_outputs)
        self.dropout21 = nn.Dropout(dropout)
        
        # 结构： 膨胀因果卷积 → 修改尺寸 → ReLU → dropout → 
        #       膨胀因果卷积 → 修改尺寸 → ReLU → dropout 
        self.net1 = nn.Sequential(self.conv11, self.chomp11, self.relu11, self.dropout11,
                                 self.conv21, self.chomp21, self.relu21, self.dropout21)
        
        # 尺度2
        padding2 = (kernel_size2-1) * dilation
        self.conv12 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size2,
                                           stride=stride, padding=padding2, dilation=dilation))
        self.chomp12 = Chomp1d(padding2)
#         self.relu12 = nn.ReLU()
#         self.relu12 = nn.LeakyReLU()
        self.relu12 = nn.PReLU(n_outputs)
        self.dropout12 = nn.Dropout(dropout)
        
        self.conv22 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size2,
                                           stride=stride, padding=padding2, dilation=dilation))
        self.chomp22 = Chomp1d(padding2)
#         self.relu22 = nn.ReLU()
#         self.relu22 = nn.LeakyReLU()
        self.relu22 = nn.PReLU(n_outputs)
        self.dropout22 = nn.Dropout(dropout)
        
        self.net2 = nn.Sequential(self.conv12, self.chomp12, self.relu12, self.dropout12,
                                 self.conv22, self.chomp22, self.relu22, self.dropout22)
        
        # 尺度3
        padding3 = (kernel_size3-1) * dilation
        self.conv13 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size3,
                                           stride=stride, padding=padding3, dilation=dilation))
        self.chomp13 = Chomp1d(padding3)
#         self.relu13 = nn.ReLU()
#         self.relu13 = nn.LeakyReLU()
        self.relu13 = nn.PReLU(n_outputs)
        self.dropout13 = nn.Dropout(dropout)
        
        self.conv23 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size3,
                                           stride=stride, padding=padding3, dilation=dilation))
        self.chomp23 = Chomp1d(padding3)
#         self.relu23 = nn.ReLU()
#         self.relu23 = nn.LeakyReLU()
        self.relu23 = nn.PReLU(n_outputs)
        self.dropout23 = nn.Dropout(dropout)
        
        self.net3 = nn.Sequential(self.conv13, self.chomp13, self.relu13, self.dropout13,
                                 self.conv23, self.chomp23, self.relu23, self.dropout23)

        # 残差链接 跨层连接的1x1卷积
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
#         self.relu = nn.ReLU()
#         self.relu = nn.LeakyReLU()
        self.relu = nn.PReLU(n_outputs)
        self.init_weights()

    #初始权值
    def init_weights(self):
        self.conv11.weight.data.normal_(0, 0.01)
        self.conv21.weight.data.normal_(0, 0.01)
        self.conv12.weight.data.normal_(0, 0.01)
        self.conv22.weight.data.normal_(0, 0.01)        
        self.conv13.weight.data.normal_(0, 0.01)
        self.conv23.weight.data.normal_(0, 0.01)
        
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out1 = self.net1(x)
        out2 = self.net2(x)
        out3 = self.net3(x)
        out = out1 + out2 + out3
        res = x if self.downsample is None else self.downsample(x)  # 残差连接
        o = self.relu(out + res)
        return o


# 时间卷积网络 主结构 网络整体
class TemporalConvNet(nn.Module):
    # num_inputs:输入数据通道数
    # num_channels: 网络结构各隐藏层即输出层 输入输出通道数
    def __init__(self, num_inputs, num_channels, kernel_size1,  kernel_size2, kernel_size3, dropout):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)  # 网络层数
        for i in range(num_levels):     # 推导各层的结构
            dilation_size = 2 ** i      # 膨胀卷积系数为2的扩张

            # 第0层即输入层的输入通道数根据原始数据的通道数确定 否则其他层从设定的网络结构中读取--即需要设定隐藏、输出层的输入输出数
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]

            # 各层就是一个TemporalBlock 膨胀系数不一样，卷积核一致
            # padding=（kernel_size-1）*dilation
            layers += [TemporalBlock(in_channels, out_channels, kernel_size1,  kernel_size2, kernel_size3, stride=1, dilation=dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)   #整体网络 由这些一系列layers连接成

    def forward(self, x):
        return self.network(x)

In [ ]:
class Model(nn.Module):
    def __init__(self, sensor_num, window_size, r, tcn_inputs_size, num_channels, kernel_size1, kernel_size2, kernel_size3, dropout, fc_note1, fc_note2, output_size):
        super(Model, self).__init__()
        self.seblock1 = SEblock(sensor_num, r)
        
        self.tcn = TemporalConvNet(tcn_inputs_size, num_channels, kernel_size1,  kernel_size2, kernel_size3,  dropout=dropout)

        self.seblock2 = SEblock(num_channels[-1],r)
        
#         self.relu2 = nn.ReLU()
#         self.relu2 = nn.LeakyReLU()
        self.relu2 = nn.PReLU(num_channels[-1])
    
        self.fc1 = nn.Linear(num_channels[-1]*(window_size+3), fc_note1)   # window_size+6这里最佳是写成TCN输出的数据的-1 dim，即是每sensor数据的长度，包括均值等
#         self.relu3 = nn.ReLU()
#         self.relu3 = nn.LeakyReLU()
        self.relu3 = nn.PReLU(fc_note1)
        
        self.fc2 = nn.Linear(fc_note1, fc_note2)
#         self.relu4 = nn.ReLU()
#         self.relu4 = nn.LeakyReLU()        
        self.relu4 = nn.PReLU(fc_note2)

        
        self.output = nn.Linear(fc_note2, output_size)
#         self.relu5 = nn.ReLU()
#         self.relu5 = nn.LeakyReLU()
        self.relu5 = nn.PReLU(output_size)


    def forward(self, x):

        x = self.seblock1(x)    # B x C x L(windowsize+6)

        out = self.tcn(x)
        
        o = self.seblock2(out)     # B x C x L(windowsize+6)

        o = self.relu2(o)
        o = torch.flatten(o, start_dim = -2, end_dim = -1)    # flatten to 1 dim

        o = self.fc1(o)    #  B x L -> B x L'
        o = self.relu3(o)

        o = self.fc2(o)    # B x L' -> B x L"
        o = self.relu4(o)

        o = self.output(o)   # B x L" -> output 1
        o = self.relu5(o)

        return o

In [ ]:
from torch.utils.data import Dataset,DataLoader
from torch.nn.utils.rnn import pad_sequence

class RulDataSet(Dataset):
    def __init__(self, sensor_data, Rul_label):
        self.sensor_data = sensor_data
        self.Rul_label = Rul_label

    def __getitem__(self, item):
        return self.sensor_data[item], self.Rul_label[item]

    def __len__(self):
        return len(self.sensor_data)

In [ ]:
# training function
def train(model, mse_loss, optimizer,lossdata, data_total, label_total):
    # train()状态 ---dropout working
    model.train()

    dataset = RulDataSet(data_total, label_total)

    # 将dataset封装为data_loader
    data_loader = DataLoader(dataset, batch_size=256, shuffle=True)
    loss_total = []

    for i, data_label in enumerate(data_loader):
        data, label = data_label
        
        data = data.to(device)
        label = label.reshape(-1,1)
        label = label.to(device)
#         print('data: {}'.format(data))
#         print('label: {}'.format(label))
        
        # 获得预测值
        out = model(data)
        
        # 数据类型有出入 进行转化
        out = out.to(torch.double)
        label = label.to(torch.double)
#         print('out: {}'.format(out))
#         print('label: {}'.format(label))

        # 代价函数
        loss = torch.sqrt(mse_loss(out, label)) 
       
        loss_total.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    lossdata.append(np.mean(loss_total))
    print('训练集RMSE:{0}'.format(np.mean(loss_total)))

    return lossdata


# testing function
def test(model, mse_loss, lossdata, test_data_total, test_label_total):
    # eval()--测试/评估状态 ---dropout不作用--即所有神经元工作
    model.eval()
    
    dataset = RulDataSet(test_data_total, test_label_total)

    # 将dataset封装为data_loader
    data_loader = DataLoader(dataset, batch_size=256, shuffle=False)
    lossi = []
    for i, data_label in enumerate(data_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data_label
        labels = labels.reshape(-1,1)
        labels = labels.to(torch.double)
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 获得预测值
        out = model(inputs)

        # 数据类型有出入 进行转化
        out = out.to(torch.double)

#         print(out)
#         print(labels)
        loss = torch.sqrt(mse_loss(out, labels))

        lossi.append(loss.item())
    
    print("测试集RMSE:{0}".format(np.mean(lossi)))
    lossdata.append(np.mean(lossi))
    
    

    return lossdata


In [ ]:
# 导入RAdam
import math
# import torch
from torch.optim.optimizer import Optimizer

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [ ]:
# 测试评估训练好的模型
"""Score 函数： h < 0 h >=0 S = Sigma(e^(-h/13)-1) or Sigma(e^(h/10)-1) """

def eval_model(model, data_total, label_total, sample_num, cyclen_list, trainset=True, Draw=True):
    # eval()状态
    model.eval()

    dataset = RulDataSet(data_total, label_total)
    # 将dataset封装为data_loader
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
    
    pred_rul = []
    true_rul = []
    lossi = 0
    Score = 0  # 计分
    accy = 0    # 正确个数
    
    for i, data_label in enumerate(data_loader):
        data, label = data_label
        data = data.to(device)
        label = label.reshape(-1,1)
        label = label.to(device)
        
        # 获得预测值
        out = model(data)

        pred_rul.append(out.item())
        true_rul.append(label.item())
        
        # 计算 score
        h = out.item() - label.item()
        lossi += h**2
        si = (math.exp((-h/13.0)) - 1) if h < 0 else (math.exp((h/10.0)) - 1)
        Score += si
        # 准确 计数
        if -13 <= h <= 10:
            accy += 1 
    print('训练集RMSE：{}'.format(np.sqrt(lossi/sample_num))) if trainset else print('测试集RMSE：{}'.format(np.sqrt(lossi/sample_num)))
    print('训练集Score：{}'.format(Score)) if trainset else print('测试集Score：{}'.format(Score))
    print('训练集正确率：{}'.format(accy/sample_num)) if trainset else print('测试集正确率：{}'.format(accy/sample_num))
    
    # 仅对sample末尾进行误差计算
    Score = 0  # 计分
    accy = 0    # 正确个数
    loss2 = 0
    result_true = []
    result_pred = []
    for No_sample in range(len(cyclen_list)):
        y_hat = pred_rul[sum(cyclen_list[:No_sample+1]) - 1]
        y_true = true_rul[sum(cyclen_list[:No_sample+1]) - 1]
        
        result_true.append(y_true)
        result_pred.append(y_hat)
        
        h = y_hat - y_true
        loss2 = loss2 + (h**2)
        si = (math.e**(-h/13.0) - 1) if h < 0 else (math.e**(h/10.0) - 1)
        Score += si
        # 准确 计数
        if -13 <= h <= 10:
            accy += 1
    print('仅对sample末尾的训练集RMSE：{}'.format(np.sqrt(loss2/len(cyclen_list))))  if trainset else print('仅对sample末尾的测试集RMSE：{}'.format(np.sqrt(loss2/len(cyclen_list))))
    print('仅对sample末尾的训练集Score：{}'.format(Score)) if trainset else print('仅对sample末尾的测试集Score：{}'.format(Score))
    print('仅对sample末尾的训练集正确率：{}'.format(accy/len(cyclen_list))) if trainset else print('仅对sample末尾的测试集正确率：{}'.format(accy/len(cyclen_list)))
    
    # 绘制预测结果图
    if not trainset:
        x_No = [No+1 for No in range(len(cyclen_list))]
        plt.title('Results')
        plt.plot(x_No, result_pred, 'blue')
        plt.plot(x_No, result_true, 'r')
        plt.legend(["Predictied","True"])    # 图例
        plt.plot(x_No, result_pred, '.b')
        plt.plot(x_No, result_true, '.r')
        plt.show()
    ####仅对sample末尾进行误差计算End
    
    # 绘图 
    if Draw:
        for No_sample in range(len(cyclen_list)):
            x = [time for time in range(cyclen_list[No_sample])]
            y_hat = pred_rul[sum(cyclen_list[:No_sample+1]) - cyclen_list[No_sample] : sum(cyclen_list[:No_sample+1])]
            y_true = true_rul[sum(cyclen_list[:No_sample+1]) - cyclen_list[No_sample] : sum(cyclen_list[:No_sample+1])]
            plt.title('No.'+str(No_sample+1)+' sample of training set') if trainset else plt.title('No.'+str(No_sample+1)+' sample of test set')
            plt.plot(x, y_hat, 'blue')      # 预测
            plt.plot(x, y_true, 'r')       # 准确
            plt.legend(["Predictied","True"])    # 图例
            plt.show()


In [ ]:
# Main Function - 2
# model
# 各层channel  -- 最后需要一致
level_channels = [8, 16]
# level_num = len(level_channels)    # level depth
sk1 = 5    # kernel size
sk2 = 3
sk3 = 2
model = Model(sensor_num=14, window_size=window_size, r=2, tcn_inputs_size=14, num_channels=level_channels,
              kernel_size1=sk1, kernel_size2=sk2, kernel_size3=sk3, dropout=0.2, fc_note1=64, fc_note2=32, output_size=1)
model.to(device)

# lossfunction
mse_loss = nn.MSELoss()

# optimizer
optimizer = RAdam(model.parameters(), lr=0.001)    
scheduler = optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.1, last_epoch=-1)

In [ ]:
# Main Function - 3
nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime)+': 开始训练模型')

lossdata_train = []
lossdata_test = []
iter_n = 100

# Runinig
for epoch in range(iter_n):
    print('epoch:', epoch)
    lossdata_train = train(model, mse_loss, optimizer, lossdata_train, train_data, train_label)
    # eval testing set data
    lossdata_test = test(model, mse_loss, lossdata_test, test_data, test_label)
    scheduler.step()
        
plt.title("loss with epoch")
plt.xlabel("epoch")
plt.xlim(0, int(iter_n))
plt.ylabel("RMSEloss")

plt.plot([i for i in range(iter_n)], lossdata_train, 'r', label='training set')
plt.plot([i for i in range(iter_n)], lossdata_test, 'b', label='testing set')
plt.legend()
plt.show()

#save trianed model
torch.save(model.state_dict(), '/kaggle/working/SEblockTCN_FD001_trained_'+str(iter_n)+'iter_'+str(nowTime0)+'.pth')

nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime)+': 训练完成，已保存模型 SongPaper_FD001_trained_'+str(iter_n)+'iter_'+str(nowTime0)+'.pth')

In [ ]:
# Main Function - 4
nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime) + ': 评估最终模型')
# training set
eval_model(model, train_data, train_label, train_sample_num, train_cyclen_list, trainset=True, Draw=False)
# testing set
eval_model(model, test_data, test_label, test_sample_num, test_cyclen_list, trainset=False, Draw=False)
nowTime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')  # 现在时间
print(str(nowTime)+': 本次训练评估实验完成')
time_end=time.time()
print('time cost',time_end-time_start,'s')